<a href="https://colab.research.google.com/github/2241031c/Night-Lights-Dissertation/blob/main/NL_17_6_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#This script will pre-process VIIRS stray light corrected DNB for both Tomintoul and the Galloway .

*change~ i edited this notebook on 10.7.24 to debug an error i was having with the galloway extension buffer. i reuploaded the extension file and buffer.

Note a second import has been conducted to import galloway extension and galloway core as two seperate shapefiles.

*   Clip the image to the geometry of the Galloway Forest and cairngorms.
*   Clip all the images in the VIIRS-DNB stray-light corrected image collection to the Galloway Forest.
*   Apply a mask and median filter
*   Export to drive as Tiff's





In [ ]:
#Install libraries
!pip install pycrs
!pip install rasterio
!pip install selenium
!pip install webdriver_manager
!pip install --upgrade geemap
!pip install osmnx


In [ ]:
# Import necessary libraries
import osmnx as ox
import geopandas as gpd
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
import ee
from geemap import Map
import folium
from folium import features
from folium import plugins
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from IPython.display import display, HTML

In [ ]:


try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [ ]:
try:
        ee.Initialize(project='###') #enter project number (###)
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

#clip to the Cairngorms national park and Galloway Forest.
when clipping to a shapefile ensure all files within the shapefile folder are uploaded in the same place (all the different file names prj, shx etc. as these will be referenced backend).

In [ ]:
#importing and reading the Galloway shapefile
#galloway_shp="/content/drive/MyDrive/NightLight/ArcGIS/FCPRODUCT_S_GALLOWAY_DARK_SKIES/FCPRODUCT_S_GALLOWAY_DARK_SKIES.shp"
#Galloway=geemap.shp_to_ee(galloway_shp).geometry()

In [ ]:
#setting the coordinated for the centre of Scotland
lat = 56.39
lon = -4.04

In [ ]:
#importing shapefile for galloway core area and galloway extension
galloway_ex_shp="/content/drive/MyDrive/NightLight/ArcGIS/Galloway_extension/Galloway_craigengillan_extension.shp"
galloway_core_shp="/content/drive/MyDrive/NightLight/ArcGIS/Galloway_core/Galloway_core_area.shp"
aoi_GA_ex=geemap.shp_to_ee(galloway_ex_shp).geometry()
aoi_GA_c=geemap.shp_to_ee(galloway_core_shp).geometry()


In [ ]:
Tomintoul_shp="/content/drive/MyDrive/NightLight/ArcGIS/Tominoul_boundary/Tomintoul_boundary.shp"
aoi_T=geemap.shp_to_ee(Tomintoul_shp).geometry()

#import files for the Tomintoul buffer.
here a 20km buffer is used, this was refined to a 10km buffer after further research.



In [ ]:
Tomintoul_inc_shp="/content/drive/MyDrive/NightLight/ArcGIS/Tominoul_boundary/Tomintoul_20km_buffer_inc/Tomintoul_20km_buffer_inc.shp"
aoi_T_inc=geemap.shp_to_ee(Tomintoul_inc_shp).geometry()

#Clip an entire ImageCollection to Galloway Core area.

In [ ]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB1 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the aoi_GA_c to clip an image
def clip_func(x):
    return x.clip(aoi_GA_c)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_GA_c)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_GA_c = viirsDNB1.map(lambda x: x.clip(aoi_GA_c))

#Clip an entire ImageCollection to Galloway Extension

In [ ]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB2 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the aoi_GA_ex to clip an image
def clip_func(x):
    return x.clip(aoi_GA_ex)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_GA_ex)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_GA_ex = viirsDNB2.map(lambda x: x.clip(aoi_GA_ex))

#Clip entire image collection to Tomintoul

In [ ]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB3 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the CA aoi to clip an image
def clip_func(x):
    return x.clip(aoi_T)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_T)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_T = viirsDNB3.map(lambda x: x.clip(aoi_T))

#Clip entire image collection to Galloway whole area and Tomintoul with buffer

In [ ]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB5 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the CA aoi to clip an image
def clip_func(x):
    return x.clip(aoi_T_inc)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_T_inc)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_T_inc = viirsDNB5.map(lambda x: x.clip(aoi_T_inc))

#Analysis to remove transient lights by applying a spatial filter.

In [ ]:
# Load the VIIRS dataset and filter by date range
def load_viirs_data(start_date, end_date, aoi):
    dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
                .filterDate(start_date, end_date)\
                .select('avg_rad')\
                .mean()\
                .clip(aoi)
    return dataset

# Apply the mask
def apply_mask(image):
    mask = image.gt(0)  # Masking non-positive values (assuming light pollution data has positive values)
    return image.updateMask(mask)

# Define a function to apply a median filter
def apply_median_filter(image, kernel_size):
    return image.focal_median(kernel_size, 'circle', 'meters')

# Load, mask, and filter datasets for each year for both AOIs
def process_data_for_aoi(start_year, end_year, aoi):
    viirs_data = {}
    for year in range(start_year, end_year + 1):
        data = apply_mask(load_viirs_data(f'{year}-01-01', f'{year}-12-31', aoi))
        filtered_data = apply_median_filter(data, kernel_size)
        viirs_data[year] = filtered_data
    return viirs_data

# Define kernel size
kernel_size = 3

# Process data for Galloway core, Galloway extension and Tomintoul
#I'm going to edit the start dates so they represent when the AOI was a dark sky park
galloway_c_data = process_data_for_aoi(2014, 2023, aoi_GA_c)
galloway_ex_data = process_data_for_aoi(2021, 2023, aoi_GA_ex)
aoi_T_data = process_data_for_aoi(2018, 2023, aoi_T)
aoi_T_inc_data = process_data_for_aoi(2018, 2023, aoi_T_inc)

Now to map

In [ ]:
#This needs to ba added to allow you to make a map with folium, it basically makes a notebook "trusted"
def folium_deepnote_show(m):
    data = m.get_root().render()
    data_fixed_height = data.replace('width: 100%;height: 100%', 'width: 100%').replace('height: 100.0%;', 'height: 609px;', 1)
    display(HTML(data_fixed_height))

In [ ]:
# Function to add an EE layer to the map
def add_ee_layer(map_obj, ee_image, vis_params, name):
    map_id_dict = ee.Image(ee_image).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(map_obj)

# Visualization parameters
vis_params = {'min': 0, 'max': 100, 'palette': ['black', 'yellow', 'orange', 'red', 'green', 'blue', 'purple']}

# Function to add a legend to the map
def add_legend(map_obj, title, colors, labels):
    legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; width: 150px; height: 200px;
    background-color: white; border:2px solid grey; padding:10px; z-index:9999; font-size:14px;">
    &nbsp; <b>{}</b> <br>
    '''.format(title)

    for color, label in zip(colors, labels):
        legend_html += '''
        &nbsp; <i style="background:{}; width:20px; height:20px; float:left; margin-right:5px;"></i> {} <br>
        '''.format(color, label)

    legend_html += '</div>'

    map_obj.get_root().html.add_child(folium.Element(legend_html))

# Define the color palette and corresponding labels
colors = ['black', 'yellow', 'orange', 'red', 'green', 'blue', 'purple']
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-100']



In [ ]:
!pip install mapbox # mapbox have some cool basemaps, could look into for presenting

In [ ]:
# Create a folium map for Galloway core and extension
galloway_map = folium.Map(location=[55.0, -4.5], zoom_start=9, tiles='Cartodb Positron')
for year, data in galloway_c_data.items():
    add_ee_layer(galloway_map, data, vis_params, str(year))
for year, data in galloway_ex_data.items():
    add_ee_layer(galloway_map, data, vis_params, str(year))
add_legend(galloway_map, 'Radiance (nW/cm2/sr)', colors, labels)
galloway_map.add_child(folium.LayerControl())
folium_deepnote_show(galloway_map)

In [ ]:
# Create a folium map for Tomintoul
aoi_T_map = folium.Map(location=[57.07, -3.55], zoom_start=9, tiles='Cartodb Positron')
for year, data in aoi_T_data.items():
    add_ee_layer(aoi_T_map, data, vis_params, str(year))
add_legend(aoi_T_map, 'Radiance (nW/cm2/sr)', colors, labels)
aoi_T_map.add_child(folium.LayerControl())
folium_deepnote_show(aoi_T_map)

In [ ]:
# Create a folium map to make sure whole buffer area is covered
aoi_Test_map = folium.Map(location=[55.0, -4.5], zoom_start=9, tiles='Cartodb Positron')
for year, data in aoi_T_inc_data.items():
    add_ee_layer(aoi_Test_map, data, vis_params, str(year))
add_legend(aoi_Test_map, 'Radiance (nW/cm2/sr)', colors, labels)
aoi_Test_map.add_child(folium.LayerControl())
folium_deepnote_show(aoi_Test_map)

#Now i will export all these to drive as Tiff's for use in ArcGIS

In [ ]:
#Load, mask, and filter datasets for a specific year for the AOI
def export_viirs_data(year, aoi, export_name):
    data = apply_mask(load_viirs_data(f'{year}-01-01', f'{year}-12-31', aoi))
    task = ee.batch.Export.image.toDrive(image=data,
                                         description=f'VIIRS_{export_name}_{year}',
                                         folder=folder_name,
                                         fileNamePrefix=f'VIIRS_{export_name}_{year}',
                                         scale=500,
                                         region=aoi)
    task.start()

    # Export data for each year for both Galloway and Cairngorms   (I reuse aois as a variable here- hash out as requires to avoid errors )
#aois = {'Galloway': Galloway, 'Cairngorms': aoi_CA}
#aois = {'Tomintoul':aoi_T}
aois ={'Tomintoul_inclusive':aoi_T_inc}
folder_name = 'VIIRS_GA_T_20km_inclusive'
for name, aoi in aois.items():
    for year in range(2014, 2023 + 1):
        export_viirs_data(year, aoi, name)

# Monitor task status
tasks = ee.batch.Task.list()
for task in tasks:
    print(task.status())

{'state': 'READY', 'description': 'VIIRS_Tomintoul_inclusive_2023', 'priority': 100, 'creation_timestamp_ms': 1720609504439, 'update_timestamp_ms': 1720609504439, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RFZR6RALCFCF4KCEHMRE4RIB', 'name': 'projects/laurabeth1418/operations/RFZR6RALCFCF4KCEHMRE4RIB'}
{'state': 'READY', 'description': 'VIIRS_Tomintoul_inclusive_2022', 'priority': 100, 'creation_timestamp_ms': 1720609504078, 'update_timestamp_ms': 1720609504078, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RB7CHDSQCZECGYJA2MXCN77S', 'name': 'projects/laurabeth1418/operations/RB7CHDSQCZECGYJA2MXCN77S'}
{'state': 'READY', 'description': 'VIIRS_Tomintoul_inclusive_2021', 'priority': 100, 'creation_timestamp_ms': 1720609503560, 'update_timestamp_ms': 1720609503560, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '4EEBUHEWA2EL4MZULWPKNIBK', 'name': 'projects/laurabeth1418/operations/4EEBUHEWA2EL4MZULWPKNIBK'}
{'state': 'READY', 'description': 'VI